In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
hate_speech = pd.read_csv('/content/drive/MyDrive/hate speech and offensive language dataset/labeled_data.csv')
list(hate_speech['tweet'])

["!!! RT @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out...",
 '!!!!! RT @mleew17: boy dats cold...tyga dwn bad for cuffin dat hoe in the 1st place!!',
 '!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby4life: You ever fuck a bitch and she start to cry? You be confused as shit',
 '!!!!!!!!! RT @C_G_Anderson: @viva_based she look like a tranny',
 '!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you hear about me might be true or it might be faker than the bitch who told it to ya &#57361;',
 '!!!!!!!!!!!!!!!!!!"@T_Madison_x: The shit just blows me..claim you so faithful and down for somebody but still fucking with hoes! &#128514;&#128514;&#128514;"',
 '!!!!!!"@__BrighterDays: I can not just sit up and HATE on another bitch .. I got too much shit going on!"',
 "!!!!&#8220;@selfiequeenbri: cause I'm tired of you big bitches coming for us skinny girls!!&#8221;",
 '" &amp; you might not get ya bitch back &amp; thats t

In [ ]:
orig_list = list(hate_speech['tweet'])
list_eng = orig_list[1000:1100]
len(list_eng)

100

In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.7 MB/s eta 0:00:00


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 62.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 22.7 MB/s eta 0:00:00


In [ ]:
import transformers
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")


In [ ]:
import csv
from collections import OrderedDict

vowels = OrderedDict([
('ँ','n'),
('ं','n'),
('ः','a'),
('अ','a'),
('आ','aa'),
('इ','i'),
('ई','ee'),
('उ','u'),
('ऊ','oo'),
('ऋ','ri'),
('ए','e'),
('ऐ','ae'),
('ओ','o'),
('औ','au'),
('ा','a'),
('ि','i'),
('ी','i'),
('ु','u'),
('ू','oo'),
('ृ','ri'),
('े','e'),
('ै','ai'),
('ो','o'),
('ौ','au')
])

consonants = OrderedDict([
('क','k'),
('ख','kh'),
('ग','g'),
('घ','gh'),
('ङ','ng'),

('च','ch'),
('छ','chh'),
('ज','j'),
('ज़','z'),
('ज़','z'), #these two are very different, see them in unicode by 'ज़'.encode('utf-8'). You'll see.
('झ','jh'),
('ञ','nj'),

('ट','t'),
('ठ','th'),
('ड','d'),
('ड़','r'),
('ड़','r'), #these two are very different, see them in unicode by 'ड़'.encode('utf-8'). You'll see.
('ढ','dh'),
('ण','n'),

('त','t'),
('थ','th'),
('द','d'),
('ध','dh'),
('न','n'),

('प','p'),
('फ','ph'),
('फ़','f'),
('फ़','f'), #these two फ़ are very different, see them in unicode by 'फ़'.encode('utf-8'). You'll see.
('ब','b'),
('भ','bh'),
('म','m'),

('य','y'),
('र','r'),
('ल','l'),
('व','v'),
('श','sh'),

('ष','sh'),
('स','s'),
('ह','h'),
('क्ष','ksh'),
('त्र','tr'),
('ज्ञ','gy')
	])

with open('/content/drive/MyDrive/devanagari-to-roman-script-transliteration-master/devanagari-to-roman-script-transliteration-master/svar.csv', 'w') as f:
	csvwriter = csv.writer(f)
	for k,v in zip(vowels.keys(), vowels.values()):
		csvwriter.writerow([k,v])

with open('/content/drive/MyDrive/devanagari-to-roman-script-transliteration-master/devanagari-to-roman-script-transliteration-master/vyanjan.csv', 'w') as f:
	csvwriter = csv.writer(f)
	for k,v in zip(consonants.keys(), consonants.values()):
		csvwriter.writerow([k,v])

In [ ]:
# -*- coding: utf-8 -*-
import csv
import sys

reader = csv.reader(open('/content/drive/MyDrive/devanagari-to-roman-script-transliteration-master/devanagari-to-roman-script-transliteration-master/svar.csv', 'r'))
vowels = {}
for row in reader:
	k, v = row
	vowels[k] = v

reader = csv.reader(open('/content/drive/MyDrive/devanagari-to-roman-script-transliteration-master/devanagari-to-roman-script-transliteration-master/vyanjan.csv', 'r'))
consonants = {}
for row in reader:
	k, v = row
	consonants[k] = v

In [ ]:
def convert_word_to_roman(content):
  str1 = ""
  for x in content:
    for y in x.split():
      for i in range(len(y)):
        if (i+1<len(y) and y[i+1].strip()==' ़'.strip()):
          c = y[i]+y[i+1]
          p=2
        else:
          c = y[i]
          p=1
        if (c in vowels.keys()):
          str1 = str1 + vowels[c]
        elif (c in consonants.keys()):
          if(i+p<len(y) and y[i+p] in consonants.keys()):
            if ((c=='झ' and i!=0) or (i!=0 and i+p+1<len(y) and y[i+p+1] in vowels.keys())): # add 'a' after 'jh', only if झ appears in the starting of the word
              str1 = str1 + consonants[c]
            else:
              str1 = str1 + consonants[c]+'a'
          else:
            str1 = str1 + consonants[c]
        elif y[i] in ['\n','\t',' ','!',',','।','-',':','\\','_','?'] or c.isalnum():
          str1 = str1 + c.replace('।','.')

  return str1

In [ ]:
def generate_hindi_from_english(sent):
  result = sent.split(" ")
  sent_tokenized = sent.split(" ")

  tokenizer.src_lang = "en_XX"
  encoded_hi = tokenizer(sent, return_tensors="pt")
  generated_tokens = model.generate(
    **encoded_hi,
    forced_bos_token_id=tokenizer.lang_code_to_id["hi_IN"]
  )
  res = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
  hindi_sub_list = res[0].split(" ")
  hindi = ""
  for i in hindi_sub_list:
    hindi = hindi + convert_word_to_roman(i) + " "
  print(hindi)
  return hindi

In [ ]:
list_hindi = []
count = 1
for i in list_eng:
  hindi = generate_hindi_from_english(i)
  print(count)
  count = count + 1
  list_hindi.append(hindi)

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (200) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


128514128514128514128514128514128514128514128514betysweetcocker: vh mkkhi sirph 128561 aaeema mano usne abhi ek bchcha tha jaise phle din 
1


/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (200) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


128514128514128514128514128514128514128514128514128514 AlisaRenee: ydi aap sooar ki chops khate hain to aap ek pagl hain 
2
128514128514128514128514128514128514128514128514128514128514128514128514128514128514 RT PushupSensei: McNair ke pas grill pr brisket sizzlin tha jb unki bajoo hoe ne uji khel ko niche rkha 
3
128514128514128514128514128557128557128557128557RT KfromtheG: meksiki bits ko moorkh puddi ki trh aakar diya gya hai! 
4
128514128514128514128514RT KfromtheG: agr tum aajibaee ko tooti huee hai to tum ek huoo ho 
5
128514128514128514RT LiViBADD88: cheap hair cheap dresses cheap shoes tacky ass little bitches 
6
128514128514128514RT kwagiheath: ve 1st 48 charloti gon nikks gon ephkek ke aaspas aur brn baink ऑph amerika stediym niche 128293 
7
12851412851420 BrandonBeSlydin: mere pagl khuda broo, dkshin ki or se pkshi mere ghr ke samne hain 
8
Austin__321 http:tcoGER0IPaBPA 
9
128514128557 RT kiaaXO_: RT sweetbre_xo: Lmfao BeingMsDaisy: WTF ek trh ki mkkhi meri man mujhe lgta h

In [ ]:
custom_df = pd.DataFrame()
custom_df['English'] = list_eng
custom_df['Hindi'] = list_hindi
custom_df.to_csv("/content/drive/MyDrive/custom_roman_hindi_dataset_11.csv")

In [ ]:
jigsaw_val = pd.read_csv('/content/drive/MyDrive/jigsaw-toxic-severity-rating/comments_to_score.csv')
jigsaw_val

,comment_id,text
0,114890,"""\n \n\nGjalexei, you asked about whether ther..."
1,732895,"Looks like be have an abuser , can you please ..."
2,1139051,I confess to having complete (and apparently b...
3,1434512,"""\n\nFreud's ideas are certainly much discusse..."
4,2084821,It is not just you. This is a laundry list of ...
...,...,...
7532,504235362,"Go away, you annoying vandal."
7533,504235566,This user is a vandal.
7534,504308177,""" \n\nSorry to sound like a pain, but one by f..."
7535,504570375,Well it's pretty fucking irrelevant now I'm un...


In [ ]:
jigsaw_val['text']

0       "\n \n\nGjalexei, you asked about whether ther...
1       Looks like be have an abuser , can you please ...
2       I confess to having complete (and apparently b...
3       "\n\nFreud's ideas are certainly much discusse...
4       It is not just you. This is a laundry list of ...
                              ...                        
7532                        Go away, you annoying vandal.
7533                             This user is a vandal.  
7534    " \n\nSorry to sound like a pain, but one by f...
7535    Well it's pretty fucking irrelevant now I'm un...
7536    The team name is Great Britain and Northern Ir...
Name: text, Length: 7537, dtype: object

In [ ]:
orig_list = list(jigsaw_val['text'])
list_eng = orig_list[250:300]
len(list_eng)

50

In [ ]:
list_eng

['"\nand I will\n\nDear Jim,\n \nI give up.  I have posted the following at page talk:cherokee.  Please unlock and DELETE all content authored by me and I will flag the pages I created for deletion.  I will notify Dr. Delso tommorrow and I will be recommending some other on-line project for us to pursue.  I am less than pleased with the handling of this site and you have not kept your word to me.  As such, I am recommending to my people to NOT support this site.   \n\nJeff V. Merkey\n\nI have been out most of the day and have reviewed what\'s gone on here on this site since I have been out with my wife and son.  I have asked Marty (mmartin) to cease in posting me to this section or this article, and to be honest, while it strokes my ego a great deal, I cannot support a sewer site like this one based upon what I have seen go on here.  I am a member of the Nation but I am not going to post private information to this site, or allow  this ""lynch mob"" mentality to go on here, nor do I ha

In [ ]:
e_l = []
for i in range(len(list_eng)):
  l_new = list_eng[i].split(".")
  for j in l_new:
    e_l.append(j)

In [ ]:
len(e_l)

491

In [ ]:
list_hindi = []
count = 1
for i in e_l:
  hindi = generate_hindi_from_english(i)
  print(count)
  count = count + 1
  list_hindi.append(hindi)

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (200) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


aur main pyara jim, main chhod doonga 
1
mainne nimnlikhit ko prishth varta pr post kiya haiacheroki 
2
kripya kholen aur mere dvara prayojit sbhi samgri mitaen aur main mitane ke lie mainne bnae ge prishthon ko phlaig krenge 
3
main da 
4
kl Delso aur main hm ke lie aage bdhne ke lie kuchh any ऑnlain priyojna sipharish krenge 
5
main is sait ke snchaln ke lie km se km khush hoon aur aap mere lie apni bat nhin rkhi hai 
6
is prkar, main apne logon ko is sait ko smrthn nhin krne ke lie sipharish krta hoon 
7
jeph vi 
8
main din ka adhikansh bhag bahr rha hoon aur is sait pr kya ho gya hai smiksha ki hai kyonki main apni ptni aur bete ke sath bahr rha hoon 
9
mainne marti marti se kha hai ki mujhe is dhara ya is lekh men post krne men bnd kr den, aur schche hone ke lie, jbki yh mere ahn ko bhut dhkelta hai, main is trh ke ek ml nikasi sait ka smrthn nhin kr skta hai is pr aadharit kya mainne dekha hai yhan chla ja rha hai 
10
main rashtr ka sdsy hoon lekin main is sait pr niji jankari nh

In [ ]:
custom_df = pd.DataFrame()
custom_df['English'] = e_l
custom_df['Hindi'] = list_hindi
custom_df.to_csv("/content/drive/MyDrive/custom_roman_hindi_dataset_12.csv")

In [ ]:
orig_list = list(jigsaw_val['text'])
list_eng = orig_list[300:350]
len(list_eng)
e_l = []
for i in range(len(list_eng)):
  l_new = list_eng[i].split(".")
  for j in l_new:
    e_l.append(j)
list_hindi = []
count = 1
for i in e_l:
  hindi = generate_hindi_from_english(i)
  print(count)
  count = count + 1
  list_hindi.append(hindi)


In [ ]:
custom_df

In [ ]:
custom_df = pd.DataFrame()
custom_df['English'] = e_l
custom_df['Hindi'] = list_hindi
custom_df.to_csv("/content/drive/MyDrive/custom_roman_hindi_dataset_13.csv")